# Extract statements for the following configurations
1) Common config: {user_inputs_file: None, num_samples: 500, random_sample: True, random_seed: 42, model: gpt-4o-mini, grounding: True}
2) Hyperparameters: model, structured_output, n_shot_prompting, self_correction_iterations, grounding

In [1]:
import sys
from pathlib import Path
import os
# get current path
sys.path.append(str(Path.cwd().parent))


In [ ]:
from time import strftime
from pathlib import Path

today_date = strftime("%Y-%m-%d")
common_config = {
    "user_inputs_file": None,
    "num_samples": 500,
    "random_sample": True,
    "random_seed": 42,
    "model": "gpt-4o-mini",
    "grounding": True
}


In [3]:
config_1 = {
    "structured_output": False,
    "n_shot_prompting": 0,
    "self_correction_iterations": 0
}
config_2 = {
    "structured_output": False,
    "n_shot_prompting": 0,
    "self_correction_iterations": 2
}
config_3 = {
    "structured_output": False,
    "n_shot_prompting": 2,
    "self_correction_iterations": 0
}
config_4 = {
    "structured_output": False,
    "n_shot_prompting": 2,
    "self_correction_iterations": 2
}
config_5 = {
    "structured_output": True,
    "n_shot_prompting": 0,
    "self_correction_iterations": 0
}
config_6 = {
    "structured_output": True,
    "n_shot_prompting": 0,
    "self_correction_iterations": 2
}
config_7 = {
    "structured_output": True,
    "n_shot_prompting": 2,
    "self_correction_iterations": 0
}
config_8 = {
    "structured_output": True,
    "n_shot_prompting": 2,
    "self_correction_iterations": 2
}


In [4]:
def get_output_file_name(common_config, specific_config, today_date_str):
    formatted_output_file_name = (
        f"stmt_extractions_model_{common_config['model']}_"
        f"structured-output_{specific_config['structured_output']}_"
        f"shot-prompting_{specific_config['n_shot_prompting']}_"
        f"self-corrections_{specific_config['self_correction_iterations']}_"
        f"{today_date_str}.pkl"
    )
    return formatted_output_file_name


In [5]:
configs = [config_1, config_2, config_3, config_4, config_5, config_6, config_7, config_8]
for config in configs:
    output_file_name = get_output_file_name(common_config, config, today_date)
    config["output_file"] = output_file_name


In [6]:
for config in configs:
    config.update(common_config)


In [7]:
for config in configs:
    print(config)


{'structured_output': False, 'n_shot_prompting': 0, 'self_correction_iterations': 0, 'output_file': 'stmt_extractions_model_gpt-4o-mini_structured-output_False_shot-prompting_0_self-corrections_0_2025-03-03.pkl', 'user_inputs_file': None, 'num_samples': 5, 'random_sample': True, 'random_seed': 42, 'model': 'gpt-4o-mini', 'grounding': True}
{'structured_output': False, 'n_shot_prompting': 0, 'self_correction_iterations': 2, 'output_file': 'stmt_extractions_model_gpt-4o-mini_structured-output_False_shot-prompting_0_self-corrections_2_2025-03-03.pkl', 'user_inputs_file': None, 'num_samples': 5, 'random_sample': True, 'random_seed': 42, 'model': 'gpt-4o-mini', 'grounding': True}
{'structured_output': False, 'n_shot_prompting': 2, 'self_correction_iterations': 0, 'output_file': 'stmt_extractions_model_gpt-4o-mini_structured-output_False_shot-prompting_2_self-corrections_0_2025-03-03.pkl', 'user_inputs_file': None, 'num_samples': 5, 'random_sample': True, 'random_seed': 42, 'model': 'gpt-4o-

In [8]:
from indra_gpt.scripts.run_statement_json_extraction import main

for config in configs:
    main(**config)


INFO: [2025-03-03 22:08:00] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
INFO: [2025-03-03 22:08:00] indra_gpt.scripts.run_statement_json_extraction - Running structured knowledge extraction pipeline...
INFO: [2025-03-03 22:08:00] indra_gpt.pipelines.statement_extraction_pipeline - Starting preprocessing step...
INFO: [2025-03-03 22:08:00] indra_gpt.processors.pre_processors - Processing user inputs...
INFO: [2025-03-03 22:08:00] indra_gpt.pipelines.statement_extraction_pipeline - Running LLM-based statement extraction...
INFO: [2025-03-03 22:08:00] indra_gpt.processors.generators - Config used for generation: {'model': 'gpt-4o-mini', 'structured_output': False, 'self_correction_iterations': 0}
Extracting:   0%|          | 0/5 [00:00<?, ?statement/s]


UnboundLocalError: cannot access local variable 'response_content' where it is not associated with a value